## Training
Here we have taken the train and test data from Faster R-CNN model so that fair comparison can be made. Hence they are also partition in 90/10 percentage split.

### Creating TFRecord
The code below generates the TFRecord file for the train and test data partitioned with the help of the *generate tfrecord.py* file.<br>
Converting to TFRecord format improves efficiency because it can take up less space than the original data and also allows for quick I/O, which is especially important when working with GPU or TPU devices.

In [2]:
!python generate_tfrecord.py -x Tensorflow/workspace/images/train -l Tensorflow/workspace/annotations/label_map.pbtxt -o Tensorflow/workspace/annotations/train.record
!python generate_tfrecord.py -x Tensorflow/workspace/images/test -l Tensorflow/workspace/annotations/label_map.pbtxt -o Tensorflow/workspace/annotations/test.record

Successfully created the TFRecord file: Tensorflow/workspace/annotations/train.record
Successfully created the TFRecord file: Tensorflow/workspace/annotations/test.record


### Download TF Models
command to clone the tensorflow models for this project so that important python files can be used such as *model_main_tf2.py* and *exporter_main_v2.py*. 

In [4]:
!cd Tensorflow && git clone https://github.com/tensorflow/models

Cloning into 'models'...
remote: Enumerating objects: 68353, done.
remote: Counting objects: 100% (60/60), done.
remote: Compressing objects: 100% (37/37), done.
remote: Total 68353 (delta 32), reused 47 (delta 22), pack-reused 68293
Receiving objects: 100% (68353/68353), 576.57 MiB | 41.06 MiB/s, done.
Resolving deltas: 100% (48021/48021), done.


### Setting Model Path

In [5]:
CUSTOM_MODEL_NAME = 'birds_ssd_mobnet'

After that:
- I manually created a directory with the same name in *Tensorflow\workspace\models\* and copied the pipeline.config file from the SSD MobileNet V1 FPN 640x640 downloaded model and pasted it into the same directory.
- Then I adjusted the configuration file manually for transfer learning, changing the number of classes to three and other parameters that were required, such as *fine tune checkpoint*, *fine tune checkpoint type*, *label map path*, *batch_size* and so on, to make a fair comparison with the Faster R-CNN model.

### Training the model
Let's train our model with the help of *model_main_tf2.py* file where I have set the training model directory and a path to *pipeline.config* file for our parameter values to consider.<br>

I started with 5000 steps and gradually increased it to 25000 before settling on 20000 steps because it was performing fine and with fair accuracy. Also, above this steps the model performance was nearly identical, and below this, the model performed was not good and also the value of loss was not satisfying enough according to the tensorboard graphs. Finally at 5000 steps, the model didn't performed well but because it was unable to detect images properly with less accuracy during inference process and also some of the images were wrongly identified.

In [6]:
# Train
!python model_main_tf2.py --model_dir=Tensorflow/workspace/models/{CUSTOM_MODEL_NAME} --pipeline_config_path=Tensorflow/workspace/models/{CUSTOM_MODEL_NAME}/pipeline.config --num_train_steps=20000 --alsologtostder

2022-01-04 21:08:45.044476: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
2022-01-04 21:08:46.630770: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcuda.so.1
2022-01-04 21:08:46.652776: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-01-04 21:08:46.653261: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1733] Found device 0 with properties: 
pciBusID: 0000:01:00.0 name: GeForce RTX 3090 computeCapability: 8.6
coreClock: 1.755GHz coreCount: 82 deviceMemorySize: 23.70GiB deviceMemoryBandwidth: 871.81GiB/s
2022-01-04 21:08:46.653281: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
2022-01-04 21:08:46.655206: I tensorflow/stream_executor/platfor

### Exporting trained inference graph
The code below helps to export the trained model to a directory with the help of *exporter_main_v2.py* file which will be used to perform object detection later.

In [7]:
!python exporter_main_v2.py --input_type image_tensor --pipeline_config_path ./Tensorflow/workspace/models/{CUSTOM_MODEL_NAME}/pipeline.config --trained_checkpoint_dir ./Tensorflow/workspace/models/{CUSTOM_MODEL_NAME}/ --output_directory ./Tensorflow/workspace/models/{CUSTOM_MODEL_NAME}/saved_model/

2022-01-04 22:18:15.868983: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
2022-01-04 22:18:17.604594: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcuda.so.1
2022-01-04 22:18:17.628975: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-01-04 22:18:17.629460: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1733] Found device 0 with properties: 
pciBusID: 0000:01:00.0 name: GeForce RTX 3090 computeCapability: 8.6
coreClock: 1.755GHz coreCount: 82 deviceMemorySize: 23.70GiB deviceMemoryBandwidth: 871.81GiB/s
2022-01-04 22:18:17.629480: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
2022-01-04 22:18:17.633488: I tensorflow/stream_executor/platfor